In [1]:
import requests


In [7]:
# retrieve download link
result = requests.post('https://europe-west3-lumpito.cloudfunctions.net/download_files', json={
    "company": "Alasco",
    "company_id": "HRB 205171",
    "documents": ["gs", "si"]
})

In [17]:
response = result.json()

# filter python list for document url with type gs
gs_doc_url = list(filter(lambda x: x["type"] == "gs", response["document_urls"]))[0]["url"]

# download document
gs_doc = requests.get(gs_doc_url)

# store document as pdf file in folder output/
with open(f"output/gs-{response['name']}.pdf", "wb") as f:
    f.write(gs_doc.content)